In [47]:
import numpy as np
import matplotlib.pyplot as plt

In [48]:
# Activation function: Sigmoid with clipping to prevent overflow
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

# Derivative of Sigmoid
def sigmoid_derivative(x):
    return x * (1 - x)

# Neural Network class with 2 hidden layers
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
        # Initialize weights and biases
        self.weights_input_hidden1 = 2 * np.random.rand(input_size, hidden1_size) - 1
        self.bias_hidden1 = 2 * np.random.rand(hidden1_size) - 1
        self.weights_hidden1_hidden2 = 2 * np.random.rand(hidden1_size, hidden2_size) - 1
        self.bias_hidden2 = 2 * np.random.rand(hidden2_size) - 1
        self.weights_hidden2_output = 2 * np.random.rand(hidden2_size, output_size) - 1
        self.bias_output = 2 * np.random.rand(output_size) - 1

    def forward(self, inputs):
        # Forward propagation
        self.hidden1_input = np.dot(inputs, self.weights_input_hidden1) + self.bias_hidden1
        self.hidden1_output = sigmoid(self.hidden1_input)
        self.hidden2_input = np.dot(self.hidden1_output, self.weights_hidden1_hidden2) + self.bias_hidden2
        self.hidden2_output = sigmoid(self.hidden2_input)
        self.output_layer_input = np.dot(self.hidden2_output, self.weights_hidden2_output) + self.bias_output
        self.output = sigmoid(self.output_layer_input)
        return self.output

    def backward(self, inputs, expected_output, learning_rate):
        # Calculate error
        output_error = expected_output - self.output
        output_delta = output_error * sigmoid_derivative(self.output)

        hidden2_error = output_delta.dot(self.weights_hidden2_output.T)
        hidden2_delta = hidden2_error * sigmoid_derivative(self.hidden2_output)

        hidden1_error = hidden2_delta.dot(self.weights_hidden1_hidden2.T)
        hidden1_delta = hidden1_error * sigmoid_derivative(self.hidden1_output)

        # Update weights and biases
        self.weights_hidden2_output += self.hidden2_output.T.dot(output_delta) * learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * learning_rate
        self.weights_hidden1_hidden2 += self.hidden1_output.T.dot(hidden2_delta) * learning_rate
        self.bias_hidden2 += np.sum(hidden2_delta, axis=0) * learning_rate
        self.weights_input_hidden1 += inputs.T.dot(hidden1_delta) * learning_rate
        self.bias_hidden1 += np.sum(hidden1_delta, axis=0) * learning_rate

        # Return the error for plotting
        return np.sum(output_error**2) / len(inputs)

    def train(self, inputs, expected_output, iterations, learning_rate):
        errors = []
        for i in range(iterations):
            self.forward(inputs)
            error = self.backward(inputs, expected_output, learning_rate)
            errors.append(error)
        return errors


In [49]:
def read_data(file_path):
    with open(file_path, 'r') as file:
        data = [line.strip().split() for line in file.readlines()]
    return np.array(data, dtype=float)

In [50]:
inputs = read_data('input.txt')
expected_output = read_data('output.txt')
expected_output = expected_output.reshape(-1, 2)  # Adjust this based on your output size

# expected_output = expected_output.reshape(-1, 1)

In [51]:
# Example usage
if __name__ == "__main__":

    # Create neural network
    nn = NeuralNetwork(input_size=2500, hidden1_size=200, hidden2_size=100, output_size=2)
    

    # Train the neural network and get the error values
    errors = nn.train(inputs, expected_output, iterations=1000, learning_rate=0.01)

    # Plot the error values / uncomment to see the plot

    # plt.plot(errors)
    # plt.xlabel('Iterations')
    # plt.ylabel('Error')
    # plt.title('Training Error over Iterations')
    # plt.show()


In [54]:
test= read_data('test.txt')

In [55]:
print("Predicted Output:")
res=nn.forward(test)
for i in range(len(res)):
    sum=res[i][0]+res[i][1]
    print(f"Circle: {(res[i][0]/sum)*100:.2f}%, Square: {(res[i][1]/sum)*100:.2f}%")
# print(res)

Predicted Output:
Circle: 99.11%, Square: 0.89%
